### Program 1
- In this assignment you will write a series of programs to solve sliding-tile puzzles using various uninformed and informed (heuristic) methods.
- **Heuristics**, or "rules of thumb," are problem-solving methods that are based on practical experience and knowledge. They allow you to use a "quick fix" to solve a minor problem or to narrow down options.
#### Part 1 – Reading and Validating Sliding-Puzzle Problems
- Read a sliding-tile puzzle problem using a JSON parser. 
- Check that the sliding-tile puzzle problem is valid. Specifically, there must be fields named *n*, *start*, and *goal*. The n field must be a positive integer greater than 1. The start and goal fields must be 𝑛 x 𝑛 matrices containing the integers 0 (for the empty space) to 𝑛2 − 1

In [238]:
import pandas as pd

df1 = pd.read_json('datasets/1-move.json')
df2 = pd.read_json('datasets/2-moves.json')
df15 = pd.read_json('datasets/15-puzzle.json')
df2


,n,start,goal
0,3,"[3, 1, 2]","[0, 1, 2]"
1,3,"[4, 0, 5]","[3, 4, 5]"
2,3,"[6, 7, 8]","[6, 7, 8]"


In [239]:
import string

def validation(df: pd.DataFrame, filename: string):
    # check if n, start and goal are defined
    if(df.columns.__contains__('n') and df.columns.__contains__('start') and df.columns.__contains__('goal')):
        # check the value of n
        if(df['n'].unique()):
            nvalue = df['n'].unique()[0]
            # if n is greater than or equal to 1, proceed
            startLength = len(df['start'])
            goalLength = len(df['goal'])
            # if the start and goal columns are of length n, proceed
            if (nvalue >= 1 and startLength == nvalue and goalLength == nvalue):
                print("'n' is greater than or equal to one. 'n' is", nvalue)
                # check the start and goal columns
                # if they are of size n AND contain a zero, proceed 
                for row,row2 in zip(df['start'],df['goal']):
                    if len(row) == nvalue and len(row2) == nvalue: 
                        if 0 in row or 0 in row2:
                            print('0 found in', row,row2)
            print(f'{filename} is valid' )
        else:
            print(f'{filename} is not valid. n is not greather than or equal to one.') 
    else:
        print(f"{filename} is not valid.JSON file does not contain fields 'n', 'start', or 'goal'")

validation(df1, '1-move.json')
validation(df2, "2-moves.json")
validation(df15, "15-puzzle.json")

'n' is greater than or equal to one. 'n' is 3
0 found in [3, 1, 2] [0, 1, 2]
0 found in [0, 4, 5] [3, 4, 5]
1-move.json is valid
'n' is greater than or equal to one. 'n' is 3
0 found in [3, 1, 2] [0, 1, 2]
0 found in [4, 0, 5] [3, 4, 5]
2-moves.json is valid
'n' is greater than or equal to one. 'n' is 4
0 found in [9, 0, 10, 7] [9, 10, 11, 12]
0 found in [13, 14, 11, 15] [13, 14, 15, 0]
15-puzzle.json is valid


#### Part 2 – Sliding-Tile Puzzle Rules
- Given a sliding-tile puzzle state, you must be able to determine the rules that are applicable to that state that can be used to generate its successor states.
- A rule has three parts:
    - *name* – a simple name for the rule (e.g., up, left, down, right)
    - *precondition function* – a Boolean function that accepts a state and returns true if the rule is applicable to state
    - *action function* – a function that accepts a state and returns the successor state obtained by applying the rule

In [279]:
from typing import List
from copy import deepcopy

def find_zero(state: pd.DataFrame): 
    for i, row in enumerate(state['start']):
        if 0 in row:
            return i, row.index(0)

def swap(state: pd.DataFrame, pos1, pos2):
    new_state = deepcopy(state['start'].tolist())   
    new_state[pos1[0]][pos1[1]], new_state[pos2[0]][pos2[1]] = new_state[pos2[0]][pos2[1]], new_state[pos1[0]][pos1[1]]
    return pd.DataFrame({'start': new_state})

def copy_state(state: pd.DataFrame):
    return state.copy()


def applicable_moves(state:pd.DataFrame) -> List[str]:
    # tuple = (index of row with 0, index of 0 in the row)
    zero_position = find_zero(state)
    zero_row = state['start'][zero_position[0]]
    moves = []

    # move right
    if zero_position[1] < len(zero_row) - 1:
        moves.append('Right') 
    # move left
    if zero_position[1] > 0:
        moves.append('Left') 
    # move up   
    if zero_position[0] > 0:
        moves.append('Up') 
    # move down
    if zero_position[0] < len(state) - 1: 
        moves.append('Down') 

    return moves

def move_and_apply(state: pd.DataFrame, direction: List[str]):
    # tuple = (index of row with 0, index of 0 in the row)
    # zero index = zero_position[1]
    # zero_row_index = zero_position[0]
    zero_position = find_zero(state)
    
    print('Starting state: \n', state)
    print()
    print('The applicable moves are: ', direction)
    print()
    for dir in direction: 

        if dir == "Right":
            new_state = copy_state(state) 
            new_zero_row = zero_position[0]
            new_zero_index = zero_position[1] + 1
            new_state = swap(new_state, zero_position, (new_zero_row, new_zero_index))
            print("Successor state after 'right'\n", new_state)
            print('\n')
        if dir == "Left":
            new_state = copy_state(state) 
            new_zero_row = zero_position[0]
            new_zero_index = zero_position[1] - 1
            new_state = swap(new_state, zero_position, (new_zero_row, new_zero_index))
            print("Successor state after 'left'\n", new_state)
            print()
        if dir == "Up":
            new_state = copy_state(state) 
            new_zero_row = zero_position[0]-1 
            new_zero_index = zero_position[1]
            new_state = swap(new_state, zero_position, (new_zero_row, new_zero_index))
            print("Successor state after 'up'\n", new_state)
            print()
        if dir == "Down":
            new_state = copy_state(state) 
            new_zero_row = zero_position[0]+1 
            new_zero_index = zero_position[1]
            new_state = swap(new_state, zero_position, (new_zero_row, new_zero_index))
            print("Successor state after 'down'\n", new_state)
            print()



In [280]:
applicable_moves(df2)

['Right', 'Left', 'Up', 'Down']

In [281]:
applicable_moves(df1)

['Right', 'Up', 'Down']

In [282]:
applicable_moves(df15)

['Right', 'Left', 'Up', 'Down']

In [283]:
move_and_apply(df2,applicable_moves(df2))

Starting state: 
    n      start       goal
0  3  [3, 1, 2]  [0, 1, 2]
1  3  [4, 0, 5]  [3, 4, 5]
2  3  [6, 7, 8]  [6, 7, 8]

The applicable moves are:  ['Right', 'Left', 'Up', 'Down']

Successor state after 'right'
        start
0  [3, 1, 2]
1  [4, 5, 0]
2  [6, 7, 8]


Successor state after 'left'
        start
0  [3, 1, 2]
1  [0, 4, 5]
2  [6, 7, 8]

Successor state after 'up'
        start
0  [3, 0, 2]
1  [4, 1, 5]
2  [6, 7, 8]

Successor state after 'down'
        start
0  [3, 1, 2]
1  [4, 7, 5]
2  [6, 0, 8]



In [284]:
move_and_apply(df1,applicable_moves(df1))

Starting state: 
    n      start       goal
0  3  [3, 1, 2]  [0, 1, 2]
1  3  [0, 4, 5]  [3, 4, 5]
2  3  [6, 7, 8]  [6, 7, 8]

The applicable moves are:  ['Right', 'Up', 'Down']

Successor state after 'right'
        start
0  [3, 1, 2]
1  [4, 0, 5]
2  [6, 7, 8]


Successor state after 'up'
        start
0  [0, 1, 2]
1  [3, 4, 5]
2  [6, 7, 8]

Successor state after 'down'
        start
0  [3, 1, 2]
1  [6, 4, 5]
2  [0, 7, 8]



In [285]:
move_and_apply(df15,applicable_moves(df15))

Starting state: 
    n             start             goal
0  4      [1, 3, 6, 4]     [1, 2, 3, 4]
1  4     [5, 2, 8, 12]     [5, 6, 7, 8]
2  4     [9, 0, 10, 7]  [9, 10, 11, 12]
3  4  [13, 14, 11, 15]  [13, 14, 15, 0]

The applicable moves are:  ['Right', 'Left', 'Up', 'Down']

Successor state after 'right'
               start
0      [1, 3, 6, 4]
1     [5, 2, 8, 12]
2     [9, 10, 0, 7]
3  [13, 14, 11, 15]


Successor state after 'left'
               start
0      [1, 3, 6, 4]
1     [5, 2, 8, 12]
2     [0, 9, 10, 7]
3  [13, 14, 11, 15]

Successor state after 'up'
               start
0      [1, 3, 6, 4]
1     [5, 0, 8, 12]
2     [9, 2, 10, 7]
3  [13, 14, 11, 15]

Successor state after 'down'
              start
0     [1, 3, 6, 4]
1    [5, 2, 8, 12]
2   [9, 14, 10, 7]
3  [13, 0, 11, 15]



### Part 3
- Implement the BACKTRACK1(DATALIST) algorithm to solve instances of the sliding-puzzle problem. The depth bound may be a global variable or passed as an argument
- Implement a main program to accept a sliding-tile puzzle problem and solve it using the BACKTRACK1(DATALIST) algorithm. Print the **start state** and the **goal state**, the **solution** and **solution length**, and the **number of states that were examined**
- Implement a main program to accept a sliding-tile puzzle problem and solve it using an *iterative depth- first search* using the BACKTRACK1(DATALIST) algorithm. Print the **cumulative number of states** examined and the **final (optimal) solution**.